In [18]:
import os
import sys
import math
import logging
from pathlib import Path

import numpy as np
import scipy as sp

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from bhm_at_scale.preprocess import gen_splits, gen_partitions, encode, make_cube
from bhm_at_scale.utils import reorder_cols

In [20]:
import kaggle

In [21]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files('pratyushakar/rossmann-store-sales', path='../data/raw', unzip=True)

In [100]:
stores = pd.read_csv('../data/raw/store.csv')
df = (pd.read_csv('../data/raw/train.csv', infer_datetime_format=True, parse_dates=['Date'])
         .assign(StateHoliday=lambda df: df.StateHoliday.map({'a': 1, 'b': 2, 'c': 3, '0': 0, 0: 0}))
         .merge(stores, how='left', on='Store')
         .sort_values(['Store', 'Date'])
         .assign(Assortment=lambda df: df.Assortment.map({'a': 1, 'b': 2, 'c': 3}))
         .assign(StoreType=lambda df: df.StoreType.map({'a': 1, 'b': 2, 'c': 3, 'd': 4}))
         .assign(StoreId=lambda df: df.groupby(['Store']).ngroup()) # make IDs contiguous
         .assign(Timestep=lambda df: df.groupby(['Store'])['Date'].cumcount())
         .assign(StoreVariant=lambda df: df['StoreType']*10 + df['Assortment'])
         .drop([c for c in stores.columns if c.startswith('Competition')], axis=1)
         .drop([c for c in stores.columns if c.startswith('Promo2S')], axis=1)
         .drop(['PromoInterval', 'StoreType', 'Assortment'], axis=1)
     )

/Users/fwilhelm/.miniconda3/envs/bhm-at-scale/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [101]:
# Use this to split stores up into 'virtual' smaller stores just to demonstrate the scalability of this approach
# df = (df.groupby('Store', as_index=False).apply(lambda df: df.assign(part=gen_partitions(len(df), 7, 22)))
#         .reset_index(drop=True)
#         .query('part >= 0')
#         .assign(Store=lambda x: x.Store*1000+x.part)
#         .drop(['part'], axis=1)
#         .sort_values(['StoreId', 'Date'])
#         .reset_index(drop=True))

In [102]:
edf = encode(df.drop(columns='Customers'), ['DayOfWeek', 'StoreVariant', 'StateHoliday'], drop_first=False)
edf = edf.drop(columns=['Date', 'Store'])
edf.loc[edf['Open'] == 0, 'Sales'] = np.nan
edf = edf.drop(columns='Open')  # Keep the open column

In [103]:
edf = reorder_cols(edf, first=['StoreId', 'Timestep'], last='Sales')

In [104]:
edf.to_csv('../data/preprocessed/edf.csv', index=False)

In [105]:
X = make_cube(edf, ['StoreId', 'Timestep'])

In [106]:
X_train, X_test = X[:1000, ...], X[1000:, ...]

In [107]:
np.savez('../data/preprocessed/X_train.npz', X_train)
np.savez('../data/preprocessed/X_test.npz', X_test)

In [108]:
df.to_csv('../data/result/df.csv', index=False)